In [1]:
# %matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import re
from matplotlib.colors import ListedColormap

from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "CandidateSummaryAction1.csv"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

CandidateSummaryAction1.csv



/Users/SamCoxon/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('CandidateSummaryAction1.csv')

value_dict = {}
dict_array = df.as_matrix()
for key in range(len(list(df.columns.values))):
    value_dict[dict_array[0][key-1]] = list(df.columns.values)[key-1]

df['tot_dis'] = df['net_ope_exp'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")
df['par_com_con'] = df['par_com_con'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")
df['ope_exp'] = df['ope_exp'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")
df['tot_con'] = df['tot_con'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")
df['ind_ref'] = df['ind_ref'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")
df['deb_owe_by_com'] = df['deb_owe_by_com'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")
df['oth_com_con'] = df['oth_com_con'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")
df['net_ope_exp'] = df['net_ope_exp'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")
df['net_con'] = df['net_con'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")
df['cas_on_han_beg_of_per']= df['cas_on_han_beg_of_per'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")
df['cas_on_han_clo_of_per']= df['cas_on_han_clo_of_per'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")
df['tot_loa']= df['tot_loa'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")
df['can_loa'] = df['can_loa'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")
df['ind_ite_con']=df['ind_ite_con'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")
df['ind_uni_con']=df['ind_uni_con'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")
df['ind_con']=df['ind_con'].str.replace("$","").str.replace(",","").str.replace("(","-").str.replace(")","")

# Erase Presidential Races
df = df.ix[~(df['can_off'] == "P")]
df = df.ix[1:]
              
df.head(3)


,can_id,can_nam,can_off,can_off_sta,can_off_dis,can_par_aff,can_inc_cha_ope_sea,can_str1,can_str2,can_cit,...,cas_on_han_beg_of_per,cas_on_han_clo_of_per,net_con,net_ope_exp,deb_owe_by_com,deb_owe_to_com,cov_sta_dat,cov_end_dat,winner,votes
1,H2GA12121,"ALLEN, RICHARD W",H,GA,12,REP,INCUMBENT,2237 PICKENS RD,NaN,AUGUSTA,...,60109.57,175613.35,1074949.50,907156.21,1125000.00,NaN,1/1/15,10/19/16,Y,158708
2,H6PA02171,"EVANS, DWIGHT",H,PA,2,DEM,CHALLENGER,PO BOX 6578,NaN,PHILADELPHIA,...,NaN,105687.23,1406719.06,1298831.83,NaN,NaN,11/2/15,10/19/16,Y,310770
3,H6FL04105,"RUTHERFORD, JOHN",H,FL,4,REP,OPEN,3817 VICKERS LAKE DRIVE,NaN,JACKSONVILLE,...,NaN,35645.09,650855.38,656210.29,60000.00,NaN,4/1/16,10/19/16,Y,286018


In [3]:
winners = df[df.winner == 'Y']
swinners = winners[winners.can_off == "S"]
hwinners = winners[winners.can_off == "H"]

hwinners.head(3)

,can_id,can_nam,can_off,can_off_sta,can_off_dis,can_par_aff,can_inc_cha_ope_sea,can_str1,can_str2,can_cit,...,cas_on_han_beg_of_per,cas_on_han_clo_of_per,net_con,net_ope_exp,deb_owe_by_com,deb_owe_to_com,cov_sta_dat,cov_end_dat,winner,votes
1,H2GA12121,"ALLEN, RICHARD W",H,GA,12,REP,INCUMBENT,2237 PICKENS RD,NaN,AUGUSTA,...,60109.57,175613.35,1074949.50,907156.21,1125000.00,NaN,1/1/15,10/19/16,Y,158708
2,H6PA02171,"EVANS, DWIGHT",H,PA,2,DEM,CHALLENGER,PO BOX 6578,NaN,PHILADELPHIA,...,NaN,105687.23,1406719.06,1298831.83,NaN,NaN,11/2/15,10/19/16,Y,310770
3,H6FL04105,"RUTHERFORD, JOHN",H,FL,4,REP,OPEN,3817 VICKERS LAKE DRIVE,NaN,JACKSONVILLE,...,NaN,35645.09,650855.38,656210.29,60000.00,NaN,4/1/16,10/19/16,Y,286018


In [ ]:
# scatter(x, y, s=20, c=None, marker='o', cmap=None, norm=None,
# vmin=None, vmax=None, alpha=None, linewidths=None, verts=None, 
# edgecolors=None, hold=None, data=None, **kwargs)
def partyAff(inp):
    if inp == "DEM":
        return 0
    elif inp == "REP": 
        return 1
    elif inp == "IND":
        return 2
    else:
        return 3
    
df['can_par_aff'] = map(partyAff, df['can_par_aff'])

sum_individual = df['tot_con']
sum_individual = sum_individual.as_matrix()
sum_individual = np.array(map(float, sum_individual))

sum_committee = df['net_ope_exp']
sum_committee = sum_committee.as_matrix()
sum_committee = np.array(map(float, sum_committee))

print sum_individual.shape

def isWinner(outcome):
    if outcome == "Y":
        return 0
    else:
        return 1
    
def isWinnerMarker(outcome):
    if outcome == "Y":
        return "*"
    else:
        return "+"

win_array = df['winner'].as_matrix()    
colors = np.array(map(isWinner, df['winner']))
print colors
# plt.scatter(sum_individual, sum_committee, s=10, c=colors, marker="+")
for index in range(len(sum_individual)):
    if win_array[index] == 'Y':
        plt.scatter(sum_individual[index], sum_committee[index], s=20, marker="o")
    else:
        plt.scatter(sum_individual[index], sum_committee[index], s=20, marker="+")
        
plt.xlabel('Total Contributions')
plt.ylabel('Net Operating Expenses')
plt.show()

In [4]:
# X = np.concatenate([[np.nan_to_num(sum_individual)],[np.nan_to_num(sum_committee)]],axis=0)
# X = X.T
# y = df['winner']
# h = 100  # step size

# cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA'])
# cmap_bold = ListedColormap(['#FF0000',
#  '#00FF00'])

# # # we create an instance of Neighbours Classifier and fit the data.
# testNN = KNeighborsClassifier(n_neighbors=15)
# testNN.fit(X, y)

# # # Plot the decision boundary. For that, we will assign a color to each
# # # point in the mesh [x_min, x_max]x[y_min, y_max].
# x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
# y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
# xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
#                      np.arange(y_min, y_max, h))
# Z = testNN.predict(np.c_[xx.ravel(), yy.ravel()])

# # # Put the result into a color plot
# Z = Z.reshape(xx.shape)
# # plt.figure()
# plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

# # # Plot also the training points
# plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold)
# plt.xlim(xx.min(), xx.max())
# plt.ylim(yy.min(), yy.max())
# plt.title("3-Class classification (k = %i, weights = '%s')"
#           % (n_neighbors, weights))

# plt.show()

# sum_individual = df['ind_con']
# sum_individual = sum_individual.as_matrix()
# sum_individual = np.array(map(float, sum_individual))

# sum_committee = df['oth_com_con']
# sum_committee = sum_committee.as_matrix()
# sum_committee = np.array(map(float, sum_committee))

# print sum_individual.shape

# def isWinner(outcome):
#     if outcome == "Y":
#         return 0
#     else:
#         return 1

# colors = np.array(map(isWinner, df['winner']))
# print colors
# plt.scatter(sum_individual, sum_committee, s=10, c=colors)
# plt.xlabel('sum_individual')
# plt.ylabel('sum_committee')
# plt.show()

In [5]:

da = df

def officeMap(inp):
    if inp == "H":
        return 0
    elif inp == "S": 
        return 100
    else:
        return 2
    
def partyAff(inp):
    if inp == "DEM":
        return 0
    elif inp == "REP": 
        return 1
    elif inp == "IND":
        return 2
    else:
        return 3
    
def seatStatus(inp):
    if inp == "INCUMBENT":
        return 0
    elif inp == "CHALLENGER": 
        return 1
    elif inp == "OPEN":
        return 2
    else:
        return 3
    
def winner(inp):
    if inp == "Y":
        return 1
    else:
        return 0
    
da['winner'] = map(winner, df['winner']) 
da['can_off'] = map(officeMap, df['can_off']) 
da['can_par_aff'] = map(partyAff, df['can_par_aff'])
da['can_inc_cha_ope_sea'] = map(seatStatus, df['can_inc_cha_ope_sea'])

features = ['Office', 'Party', 'Challenger_Status', 
            'Sum_200_Plus_Contribution', 'Sum_200_Or_Less', 
            'Sum_Individual_Contributions', 'Party_Contribution',
           'Other_Committee_Contributions', 'Total_Contributions',
           'Operating_Expenditure', 'Individual_Refund',
           'Sum_Disbursement', 'Cash_At_Start', 'Cash_At_Finish',
           'Net_Operating_Expendiure', 'Debt_Owed_By_Committee']

testset = pd.DataFrame(np.random.randn(len(da['can_off']), 
                            len(features)), columns=features)

for key in features:
    testset[key] = da[value_dict[key]]
    
testset.head(5)

,Office,Party,Challenger_Status,Sum_200_Plus_Contribution,Sum_200_Or_Less,Sum_Individual_Contributions,Party_Contribution,Other_Committee_Contributions,Total_Contributions,Operating_Expenditure,Individual_Refund,Sum_Disbursement,Cash_At_Start,Cash_At_Finish,Net_Operating_Expendiure,Debt_Owed_By_Committee
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1.0,0.0,554305.00,46969.50,601274.50,NaN,473675.00,1074949.50,908518.98,NaN,907156.21,60109.57,175613.35,907156.21,1125000.00
2,0.0,0.0,1.0,1042280.38,72430.64,1114711.02,NaN,302834.20,1417545.22,1300557.53,9326.16,1298831.83,NaN,105687.23,1298831.83,NaN
3,0.0,1.0,2.0,529030.38,13075.00,542105.38,NaN,106050.00,650855.38,656642.76,NaN,656210.29,NaN,35645.09,656210.29,60000.00
4,0.0,1.0,0.0,2479616.45,1837715.13,4317331.58,3545.32,660038.51,4980915.41,5073110.33,38471.67,5055942.15,83815.78,399122.61,5055942.15,NaN


In [6]:
# Implement Ball Tree Nearest Neighbors
k=20

# def notNan(inp):
#     return not np.isnan(inp)

y = np.nan_to_num(da['winner'].as_matrix())
X = testset.as_matrix()
X = [[float(i) for i in j] for j in X]
X = [[np.nan_to_num(i) for i in j] for j in X]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=0)

nbors = KNeighborsClassifier(n_neighbors=10)
nbors.fit(X_train, y_train)
print nbors.score(X_test, y_test)

score_array = []

for i in range(1,k):
    nbors = KNeighborsClassifier(n_neighbors=i)
    nbors.fit(X_train, y_train)
    score = nbors.score(X_test, y_test)
    score_array.append(score)
    
plt.scatter(range(1,k), score_array)
plt.show

0.943103448276


<function matplotlib.pyplot.show>

In [7]:
logReg = LogisticRegression()
logReg.fit(X_train,y_train)
print logReg.score(X_test, y_test)

0.889655172414


In [9]:
max_score_NN = (0,0,0)
max_score_Log = (0,0)
for i in range(10,25):
    nbors = KNeighborsClassifier(n_neighbors=i)
    logReg = LogisticRegression()
    for j in range(2,10):
        scores_NN = cross_val_score(nbors, X, y, cv=j)
        if(scores_NN.mean() > max_score_NN[0]):
            max_score_NN = (scores_NN.mean(), j, i)
        scores_Log = cross_val_score(logReg, X, y, cv=j)
        if(scores_Log.mean() > max_score_Log[0]):
            max_score_Log = (scores_Log.mean(), j)
            
print max_score_NN
print max_score_Log

(0.93173051241522142, 7, 10)
(0.88118435613089929, 9)
